In [20]:
import numpy as np

from scipy.stats import multivariate_normal as mvn
from numpy.linalg import inv

In [21]:
# Suppose we have a certain n-dimensional vector x. in our case n will equal 3 
true_x = [100,200,300]

In [22]:
# Let's create a simple linear function that takes ndimensional vector as it's input with parameters A and b
def f(A,b,x):
    return A@x + b

In [23]:
# We can evaluate this function on x with certain parameters
A = np.array([[5,0,0],
              [0,2,0],
              [0,0,3]])

b = np.array([10,20,30])

true_y = f(A,b,true_x)

true_y

array([510, 420, 930])

In [24]:
# Now let's suppose y is a value that we can measure m times with normal measurement uncertainty and known covariance matrix

m = 10

S = np.array([[250,40,20],
              [40,250,24],
              [20,24,250]])

# y_samples will serve as a set of example measurements
y_samples = mvn.rvs(mean=true_y, cov=S, size=m,random_state = 1)

print(true_y)
y_samples.mean(axis=0)

[510 420 930]


array([504.27767668, 423.17771858, 926.61763235])

In [25]:
# let's estimate the initial x value using maximum a posteriori estimate with know f function parameters(A and b)
# and uncertainty covariance matrix
# fortunately there's a known formula for this

In [26]:
def normal_posterior(m_prior , S_prior , S_likelyhood, A, b, y):
    # Let's calculate covariance matrix for multivariate normal posterior
    S_posterior = inv( inv(S_prior) + A.T @ inv(S_likelyhood) @ A )

    # Let's calculate mean value for multivariate normal posterior
    m_posterior = S_posterior @ (A.T@inv(S_likelyhood)@(y-b) + inv(S_prior)@m_prior)
    
    return m_posterior, S_posterior

def calculate_from_samples(y_samples, m_prior, S_prior, S_likelyhood, A, b, y):
    for y in y_samples:
            m_prior, S_prior = normal_posterior(m_prior,S_prior, S, A, b, y)

In [27]:
m_prior = [0,0,0]
S_prior = np.array([[400,0,0],
                    [0,400,0],
                    [0,0,400]])

for y_ in y_samples:
    m_prior, S_prior = normal_posterior(m_prior,S_prior, S, A, b, y_)
    
m_prior

array([ 98.31279399, 198.09867266, 296.58207676])

In [31]:
# Let's compare the result to regular linear transformation
y_mean = y_samples.mean(axis=0)
y_mean
x_freom_mean = inv(A)@(y_mean-b)
x_freom_mean

array([ 98.85553534, 201.58885929, 298.87254412])

In [29]:
# we can see that using multivariate normal posterior isn't a very effective option 
# as we get very similar results by simply calculating the median
# it's just a simple exercise of bayesian statistic 
# as we're generating an 'a posteriori' distribution from 'a priori' and other parameters